In [2]:
%pip install transformers langchain langchain-huggingface langchain-community faiss-cpu huggingface-hub unstructured gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 19.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 23.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/9

# llm instance

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_hf = user_secrets.get_secret("hf_api_key")


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=512,
    temperature=0.2,
    huggingfacehub_api_token=secret_value_hf,
)
#llm_chain = prompt | llm
#print(llm_chain.invoke({"question": question}))

# embedding model

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=secret_value_hf,
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)
#texts = ["Hello, world!", "How are you?"]
#hf_embeddings.embed_documents(texts)

# load pdf and build faiss vector database

In [ ]:
!mkdir md_files

In [ ]:
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite1/1_1733975053.267784.md" "md_files/1.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite2/2_1733975093.5062084.md" "md_files/2.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite3/3_1733975145.179506.md" "md_files/3.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite4/Adv Funct Materials - 2023 - Yu - Chalcogenide Perovskite Thin Films with Controlled Phases for Optoelectronics_1733974706.4877486.md" "md_files/4.md"

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import glob
import gradio as gr

In [ ]:
md_path = glob.glob( "md_files/*.md")

docs = [UnstructuredMarkdownLoader(md).load() for md in md_path]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)


# Create the embeddings + retriever

db = FAISS.from_documents(doc_splits,
                          hf_embeddings)

# retrieval and generation

In [ ]:
prompt_template = '''You are an assistant for question-answering tasks. 
        Here is the context to use to answer the question:
        {context} 
        Think carefully about the above context. 
        Now, review the user question:
        {question}
        Provide an answer to this questions using only the above context. 
        keep the answer concise.
        Answer:'''

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


llm_chain = prompt | llm | StrOutputParser()

In [ ]:
retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
#questions="what is advantage of BaZrS3?"
#rag_chain.invoke(questions)

# gradio interface

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import glob
import gradio as gr

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=secret_value_hf,
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)

md_path = glob.glob( "md_files/*.md")

docs = [UnstructuredMarkdownLoader(md).load() for md in md_path]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)


# Create the embeddings + retriever

db = FAISS.from_documents(doc_splits,
                          hf_embeddings)


In [ ]:

# prompt
prompt_template = '''You are an assistant for question-answering tasks. 
        Here is the context to use to answer the question:
        {context} 
        Think carefully about the above context. 
        Now, review the user question:
        {question}
        Provide an answer to this questions using only the above context. 
        Use three sentences maximum and keep the answer concise.
        Answer:'''

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# gradio interface

def get_output(model_name:str,is_RAG:str,questions:str):
    if model_name=="mistralai/Mistral-7B-Instruct-v0.2":
        #repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever(
            search_type="similarity",
            search_kwargs={'k': 4}
        )
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1
    elif model_name=="meta-llama/Llama-3.2-3B-Instruct":
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever()
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1
    elif model_name=="Qwen/Qwen2.5-72B-Instruct":
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever()
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1

demo = gr.Interface(
    fn=get_output,
    inputs=[
        gr.Radio(
            choices=["mistralai/Mistral-7B-Instruct-v0.2", "meta-llama/Llama-3.2-3B-Instruct","Qwen/Qwen2.5-72B-Instruct"],
            type="value",
            value="mistralai/Mistral-7B-Instruct-v0.2",  # Set default value to "Model 1"
            label="model name"
        ),
        gr.Radio(
            choices=["RAG", "No RAG"],
            type="value",
            value="RAG",  # Set default value to "Model 1"
            label="RAG or not"
        ),
        gr.Textbox(label="Input Questions")#,info="input questions on chalcogenide perovskites")
    ],
    outputs="markdown",
    title="Ask questions on chalcogenide perovskites",
    description="""
    ## this space is implemented by Retrieval-Augmented Generation of Large Language Models , based haolei Hui's work on chalcogenide perovskite papers.
    """,
    flagging_mode="never",
    examples=[["mistralai/Mistral-7B-Instruct-v0.2","RAG","what is advantage of BaZrS3?"],
              ["mistralai/Mistral-7B-Instruct-v0.2","RAG","what is bandgap of SrHfS3?"],
              ["mistralai/Mistral-7B-Instruct-v0.2","RAG","why is chalcogenide perovskite important?"]
              ]
)

# Launch the Gradio app
if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
# Custom CSS to style the output area
custom_css = """
#output_area {
    background-color: #1e1e1e; /* Dark background */
    color: #ffffff; /* White text */
    padding: 10px;
    border-radius: 5px;
    border: 1px solid #333333; /* Dark border */
    margin-top: 10px;
}

#output_area h3 {
    color: #ffcc00; /* Yellow title color */
    margin-bottom: 10px;
}
"""

with gr.Blocks(title="Ask Questions on Chalcogenide Perovskites",theme=gr.themes.Ocean(),css=custom_css) as demo:
    gr.Markdown("""
    ## Retrieval-Augmented Generation for Chalcogenide Perovskites
    This space implements Retrieval-Augmented Generation (RAG) using large language models, based on Hui Haolei's work on chalcogenide perovskite papers. You can select different models and choose whether to use RAG to enhance the responses.
    For details of this project, check my [github](https://github.com/HaoleiH/AI-driven-projects).
    """)
    
    with gr.Row():
        model_name = gr.Radio(
            choices=["mistralai/Mistral-7B-Instruct-v0.2", "meta-llama/Llama-3.2-3B-Instruct", "Qwen/Qwen2.5-72B-Instruct"],
            value="mistralai/Mistral-7B-Instruct-v0.2",
            label="Model Name",
            info="Select the model you want to use."
        )
    
    with gr.Row():
        rag = gr.Radio(
            choices=["RAG", "No RAG"],
            value="RAG",
            label="RAG or Not",
            info="Choose whether to use Retrieval-Augmented Generation."
        )
    
    with gr.Row():
        question = gr.Textbox(
            label="Input Question",
            placeholder="Enter your question about chalcogenide perovskites here...",
            lines=2  # Increase the number of lines for better input experience
        )
    
    with gr.Row():
        submit_button = gr.Button("Submit")
    
    with gr.Row():
        output = gr.Markdown(label="Answer",
                            #lines=10,  # Increase the number of lines for the output area
                            elem_id="output_area"  # Assign a custom ID for styling
                            )
    submit_button.click(
        fn=get_output,
        inputs=[model_name, rag, question],
        outputs=output
    )

    gr.Examples(
        examples=[
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "What is the advantage of BaZrS3?"],
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "What is the bandgap of SrHfS3?"],
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "Why is chalcogenide perovskite important?"]
        ],
        fn=get_output,
        inputs=[model_name, rag, question],
        outputs=output
    )

demo.launch()

# Final  code

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_hf = user_secrets.get_secret("hf_api_key")

In [5]:
!mkdir md_files
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite1/1_1733975053.267784.md" "md_files/1.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite2/2_1733975093.5062084.md" "md_files/2.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite3/3_1733975145.179506.md" "md_files/3.md"
!cp "/kaggle/input/cp-papers/chalcogenide_perovskite4/Adv Funct Materials - 2023 - Yu - Chalcogenide Perovskite Thin Films with Controlled Phases for Optoelectronics_1733974706.4877486.md" "md_files/4.md"

In [7]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
import glob
import gradio as gr

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=secret_value_hf,
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)

md_path = glob.glob( "md_files/*.md")

docs = [UnstructuredMarkdownLoader(md).load() for md in md_path]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)


# Create the embeddings + retriever

db = FAISS.from_documents(doc_splits,
                          hf_embeddings)


# prompt
prompt_template = '''You are an assistant for question-answering tasks. 
        Here is the context to use to answer the question:
        {context} 
        Think carefully about the above context. 
        Now, review the user question:
        {question}
        Provide an answer to this questions using only the above context. 
        Use three sentences maximum and keep the answer concise.
        Answer:'''

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# gradio interface

def get_output(model_name:str,is_RAG:str,questions:str):
    if model_name=="mistralai/Mistral-7B-Instruct-v0.2":
        #repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever(
            search_type="similarity",
            search_kwargs={'k': 4}
        )
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1
    elif model_name=="meta-llama/Llama-3.2-3B-Instruct":
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever()
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1
    elif model_name=="Qwen/Qwen2.5-72B-Instruct":
        llm = HuggingFaceEndpoint(
            repo_id=model_name,
            max_length=512,
            temperature=0.2,
            huggingfacehub_api_token=secret_value_hf,
        )
        llm_chain = prompt | llm | StrOutputParser()
        retriever = db.as_retriever()
        
        rag_chain = (
         {"context": retriever, "question": RunnablePassthrough()}
            | llm_chain
        )
        if is_RAG== "RAG":
            generation2=rag_chain.invoke(questions)
            return generation2
        else:
            generation1=llm_chain.invoke({"context":"", "question": questions})
            return generation1



# Custom CSS to style the output area
custom_css = """
#output_area {
    background-color: #1e1e1e; /* Dark background */
    color: #ffffff; /* White text */
    padding: 10px;
    border-radius: 5px;
    border: 1px solid #333333; /* Dark border */
    margin-top: 10px;
}

#output_area h3 {
    color: #ffcc00; /* Yellow title color */
    margin-bottom: 10px;
}
"""

with gr.Blocks(title="Ask Questions on Chalcogenide Perovskites",theme=gr.themes.Ocean(),css=custom_css) as demo:
    gr.Markdown("""
    ## Retrieval-Augmented Generation for Chalcogenide Perovskites
    This space implements Retrieval-Augmented Generation (RAG) using large language models, based on Hui Haolei's work on chalcogenide perovskite papers. You can select different models and choose whether to use RAG to enhance the responses.
    """)
    
    with gr.Row():
        model_name = gr.Radio(
            choices=["mistralai/Mistral-7B-Instruct-v0.2", "meta-llama/Llama-3.2-3B-Instruct", "Qwen/Qwen2.5-72B-Instruct"],
            value="mistralai/Mistral-7B-Instruct-v0.2",
            label="Model Name",
            info="Select the model you want to use."
        )
    
    with gr.Row():
        rag = gr.Radio(
            choices=["RAG", "No RAG"],
            value="RAG",
            label="RAG or Not",
            info="Choose whether to use Retrieval-Augmented Generation."
        )
    
    with gr.Row():
        question = gr.Textbox(
            label="Input Question",
            placeholder="Enter your question about chalcogenide perovskites here...",
            lines=2  # Increase the number of lines for better input experience
        )
    
    with gr.Row():
        submit_button = gr.Button("Submit")
    
    with gr.Row():
        output = gr.Textbox(label="Response",
                            lines=10,  # Increase the number of lines for the output area
                            elem_id="output_area"  # Assign a custom ID for styling
                            )
    submit_button.click(
        fn=get_output,
        inputs=[model_name, rag, question],
        outputs=output
    )

    gr.Examples(
        examples=[
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "What is the advantage of BaZrS3?"],
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "What is the bandgap of SrHfS3?"],
            ["mistralai/Mistral-7B-Instruct-v0.2", "RAG", "Why is chalcogenide perovskite important?"]
        ],
        fn=get_output,
        inputs=[model_name, rag, question],
        outputs=output
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://a038d42a34d9e9bee3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
